In [5]:
TaskObject = " \
{\
    \"TaskInstanceId\": 28,\
    \"TaskMasterId\": 2,\
    \"TaskStatus\": \"InProgress\",\
    \"TaskType\": \"Execute Synapse Notebook\",\
    \"Enabled\": 1,\
    \"ExecutionUid\": \"f3156876-086d-4915-b1b1-0b445810142c\",\
    \"NumberOfRetries\": 0,\
    \"DegreeOfCopyParallelism\": 1,\
    \"KeyVaultBaseUrl\": \"https://ads-dev-kv-ads-we7y.vault.azure.net/\",\
    \"ScheduleMasterId\": \"-4\",\
    \"TaskGroupConcurrency\": \"10\",\
    \"TaskGroupPriority\": 0,\
    \"TaskExecutionType\": \"ADF\",\
    \"ExecutionEngine\": {\
        \"EngineId\": -2,\
        \"EngineName\": \"adsdevsynwadswe7y\",\
        \"SystemType\": \"Synapse\",\
        \"ResourceGroup\": \"gfd1\",\
        \"SubscriptionId\": \"035a1364-f00d-48e2-b582-4fe125905ee3\",\
        \"ADFPipeline\": \"GPL_SparkNotebookExecution_Azure\",\
        \"TaskDatafactoryIR\": \"Azure\",\
        \"JsonProperties\": {\
            \"endpoint\": \"https://adsdevsynwadswe7y.dev.azuresynapse.net\",\
            \"DeltaProcessingNotebook\": \"DeltaProcessingNotebook\",\
            \"PurviewAccountName\": \"adsdevpuradswe7y\",\
            \"DefaultSparkPoolName\": \"adsdevsynspads\"\
        }\
    },\
    \"Source\": {\
        \"System\": {\
            \"SystemId\": -4,\
            \"SystemServer\": \"https://adsdevdlsadswe7yadsl.dfs.core.windows.net\",\
            \"AuthenticationType\": \"MSI\",\
            \"Type\": \"ADLS\",\
            \"Username\": null,\
            \"Container\": \"datalakeraw\"\
        },\
        \"Instance\": {\
            \"SourceRelativePath\": \"/samples/sif/SifOpenApi/\"\
        },\
        \"DataFileName\": \"jsonSchemaCreate_AU.json\",\
        \"RelativePath\": \"/samples/sif/SifOpenApi/\",\
        \"SchemaFileName\": \"\",\
        \"Type\": \"Notebook-Optional\",\
        \"WriteSchemaToPurview\": \"Disabled\"\
    },\
    \"Target\": {\
        \"System\": {\
            \"SystemId\": -10,\
            \"SystemServer\": \"adsdevsynwadswe7y.sql.azuresynapse.net\",\
            \"AuthenticationType\": \"MSI\",\
            \"Type\": \"Azure Synapse\",\
            \"Username\": null,\
            \"Workspace\": \"adsdevsynwadswe7y\",\
            \"Database\": \"adsdevsyndpads\"\
        },\
        \"Instance\": {\
            \"SourceRelativePath\": \"/samples/sif/SifOpenApi/\"\
        },\
        \"DataFileName\": \"\",\
        \"RelativePath\": \"\",\
        \"SchemaFileName\": \"\",\
        \"Type\": \"Notebook-Optional\",\
        \"WriteSchemaToPurview\": \"Disabled\"\
    },\
    \"TMOptionals\": {\
        \"CustomDefinitions\": \"SparkDatabaseName=SIF\",\
        \"ExecuteNotebook\": \"SIFLoadCodeSets\",\
        \"Purview\": \"Disabled\",\
        \"QualifiedIDAssociation\": \"TaskMasterId\",\
        \"UseNotebookActivity\": \"Enabled\"\
    }\
}" 

print(TaskObject)



StatementMeta(adsdevsynspads, 7, 5, Finished, Available)

 {    "TaskInstanceId": 28,    "TaskMasterId": 2,    "TaskStatus": "InProgress",    "TaskType": "Execute Synapse Notebook",    "Enabled": 1,    "ExecutionUid": "f3156876-086d-4915-b1b1-0b445810142c",    "NumberOfRetries": 0,    "DegreeOfCopyParallelism": 1,    "KeyVaultBaseUrl": "https://ads-dev-kv-ads-we7y.vault.azure.net/",    "ScheduleMasterId": "-4",    "TaskGroupConcurrency": "10",    "TaskGroupPriority": 0,    "TaskExecutionType": "ADF",    "ExecutionEngine": {        "EngineId": -2,        "EngineName": "adsdevsynwadswe7y",        "SystemType": "Synapse",        "ResourceGroup": "gfd1",        "SubscriptionId": "035a1364-f00d-48e2-b582-4fe125905ee3",        "ADFPipeline": "GPL_SparkNotebookExecution_Azure",        "TaskDatafactoryIR": "Azure",        "JsonProperties": {            "endpoint": "https://adsdevsynwadswe7y.dev.azuresynapse.net",            "DeltaProcessingNotebook": "DeltaProcessingNotebook",            "PurviewAccountName": "adsdevpuradswe7y",            "DefaultSp

**Process the TaskObject to Extract Required Information**

In [10]:
import random
import json
from pyspark.sql import Row
from pyspark.sql.types import *
from pyspark.sql.functions import *

TaskDict = {}
OutputDict = {}
TaskObjectJson = json.loads(TaskObject)

##we want to delete EngineJson as it causes issues when converting back to a json and it is not needed as its properties are within JsonProperties as children
try:
    del TaskObjectJson['ExecutionEngine']['EngineJson']
except:
    print("No EngineJson Found")

Source = TaskObjectJson['Source']['System']['Container'] + "@" + TaskObjectJson['Source']['System']['SystemServer'].replace("https://","").replace("blob","dfs") + "/"
Source = Source.replace('///', '/')
Source = Source.replace('//', '/')
Source = Source + TaskObjectJson['Source']["Instance"]['SourceRelativePath']
Source = Source  +"/" + TaskObjectJson['Source']['DataFileName'] 
Source = Source.replace('///', '/')
Source = Source.replace('//', '/')
Source = "abfss://" + Source 
print(Source)

SifDbName = "sif"
tmopts = TaskObjectJson['TMOptionals']['CustomDefinitions'].split(",")
for o in tmopts:
    opt = o.split("=")
    if (opt[0] == "SparkDatabaseName"):
        SifDbName = opt[1].lower()
        break

print(SifDbName)


StatementMeta(adsdevsynspads, 7, 10, Finished, Available)

No EngineJson Found
abfss://datalakeraw@adsdevdlsadswe7yadsl.dfs.core.windows.net/samples/sif/SifOpenApi/jsonSchemaCreate_AU.json
sif


**Create the SIF Database**

In [29]:
#check if the specified DB / table exists - if so only do required actions.
dbList = spark.catalog.listDatabases()
dbExists = False
for db in dbList:
    if (db.name == SifDbName):
        dbExists = True
        break

if (dbExists):
    print("DB Exists")    
else:
    print("DB Doesnt exist - creating DB")
    createDBString = "CREATE DATABASE " + SifDbName 
    spark.sql(createDBString)
    spark.catalog.setCurrentDatabase(SifDbName)

StatementMeta(adsdevsynspads, 1, 29, Finished, Available)

DB Exists


**Read in Full Json Schema from SIF Open API Definition File**

In [30]:
import json
rdd =  spark.sparkContext.wholeTextFiles(Source)
text = rdd.collect()[0][1]
dict1 = json.loads(str(text))

StatementMeta(adsdevsynspads, 1, 30, Finished, Available)

**Declare Function that will register codesets as managed tables**

In [31]:
from pyspark.sql import SparkSession
from pyspark.sql.types  import *
from pyspark.sql import Row

def PersistRefData(name, dict1):
    ref = dict1['definitions'][name]['oneOf']
    schema = StructType([StructField('const', StringType()), StructField('title',StringType())])
    refdf = spark.createDataFrame(data=ref, schema = schema)
    tablename = "ref_" + name
    spark.catalog.setCurrentDatabase(SifDbName)
    refdf.write.mode("overwrite").saveAsTable(tablename)

StatementMeta(adsdevsynspads, 1, 31, Finished, Available)

**Call Function for Each Codeset Entity**

In [32]:
entities = ["AbstractContentElement" \
,"AUCodeSets0211ProgramAvailability" \
,"AUCodeSets0792IdentificationProcedure" \
,"AUCodeSetsAccompaniment" \
,"AUCodeSetsACStrand" \
,"AUCodeSetsActivityInvolvementCode" \
,"AUCodeSetsActivityType" \
,"AUCodeSetsAddressRole" \
,"AUCodeSetsAddressType" \
,"AUCodeSetsAGCollection" \
,"AUCodeSetsAGContextQuestion" \
,"AUCodeSetsAGSubmissionStatus" \
,"AUCodeSetsAssessmentReportingMethod" \
,"AUCodeSetsAssessmentType" \
,"AUCodeSetsAttendanceCode" \
,"AUCodeSetsAttendanceStatus" \
,"AUCodeSetsAustralianCitizenshipStatus" \
,"AUCodeSetsAustralianStandardClassificationOfCulturalAndEthnicGroupsASCCEG" \
,"AUCodeSetsAustralianStandardClassificationOfLanguagesASCL" \
,"AUCodeSetsAustralianStandardClassificationOfReligiousGroupsASCRG" \
,"AUCodeSetsAustralianStandardGeographicalClassificationASGC" \
,"AUCodeSetsAustralianTimeZone" \
,"AUCodeSetsBirthdateVerification" \
,"AUCodeSetsBoarding" \
,"AUCodeSetsCalendarEvent" \
,"AUCodeSetsContactMethod" \
,"AUCodeSetsDayValueCode" \
,"AUCodeSetsDetentionCategory" \
,"AUCodeSetsDwellingArrangement" \
,"AUCodeSetsEducationAgencyType" \
,"AUCodeSetsEducationLevel" \
,"AUCodeSetsElectronicIdType" \
,"AUCodeSetsEmailType" \
,"AUCodeSetsEmploymentType" \
,"AUCodeSetsEnglishProficiency" \
,"AUCodeSetsEnrollmentTimeFrame" \
,"AUCodeSetsEntryType" \
,"AUCodeSetsEquipmentType" \
,"AUCodeSetsEventCategory" \
,"AUCodeSetsEventSubCategory" \
,"AUCodeSetsExitWithdrawalStatus" \
,"AUCodeSetsExitWithdrawalType" \
,"AUCodeSetsFederalElectorate" \
,"AUCodeSetsFFPOSStatusCode" \
,"AUCodeSetsFTPTStatusCode" \
,"AUCodeSetsGroupCategoryCode" \
,"AUCodeSetsImmunisationCertificateStatus" \
,"AUCodeSetsIndigenousStatus" \
,"AUCodeSetsLanguageType" \
,"AUCodeSetsLearningStandardItemRelationshipTypes" \
,"AUCodeSetsMaritalStatusAIHW" \
,"AUCodeSetsMediumOfInstruction" \
,"AUCodeSetsNameUsageType" \
,"AUCodeSetsNAPJurisdiction" \
,"AUCodeSetsNAPParticipationCode" \
,"AUCodeSetsNAPResponseCorrectness" \
,"AUCodeSetsNAPTestDomain" \
,"AUCodeSetsNAPTestItemMarkingType" \
,"AUCodeSetsNAPTestItemType" \
,"AUCodeSetsNAPTestType" \
,"AUCodeSetsNAPWritingGenre" \
,"AUCodeSetsNonSchoolEducation" \
,"AUCodeSetsOperationalStatus" \
,"AUCodeSetsPermanentResidentStatus" \
,"AUCodeSetsPermissionCategoryCode" \
,"AUCodeSetsPersonalisedPlan" \
,"AUCodeSetsPictureSource" \
,"AUCodeSetsPNPCode" \
,"AUCodeSetsPrePrimaryHours" \
,"AUCodeSetsProgramFundingSourceCode" \
,"AUCodeSetsProgressLevel" \
,"AUCodeSetsPublicSchoolCatchmentStatus" \
,"AUCodeSetsReceivingLocationOfInstruction" \
,"AUCodeSetsRelationshipToStudent" \
,"AUCodeSetsResourceUsageContentType" \
,"AUCodeSetsScheduledActivityType" \
,"AUCodeSetsSchoolCoEdStatus" \
,"AUCodeSetsSchoolEducationLevelType" \
,"AUCodeSetsSchoolEnrollmentType" \
,"AUCodeSetsSchoolFocusCode" \
,"AUCodeSetsSchoolLevel" \
,"AUCodeSetsSchoolLocation" \
,"AUCodeSetsSchoolSectorCode" \
,"AUCodeSetsSchoolSystem" \
,"AUCodeSetsSessionType" \
,"AUCodeSetsSexCode" \
,"AUCodeSetsSourceCodeType" \
,"AUCodeSetsStaffActivity" \
,"AUCodeSetsStaffStatus" \
,"AUCodeSetsStandardAustralianClassificationOfCountriesSACC" \
,"AUCodeSetsStateTerritoryCode" \
,"AUCodeSetsStudentFamilyProgramType" \
,"AUCodeSetsSuspensionCategory"  \
,"AUCodeSetsSystemicStatus" \
,"AUCodeSetsTeacherCoverCredit" \
,"AUCodeSetsTeacherCoverSupervision" \
,"AUCodeSetsTelephoneNumberType" \
,"AUCodeSetsTravelMode" \
,"AUCodeSetsVisaStudyEntitlement" \
,"AUCodeSetsVisaSubClass" \
,"AUCodeSetsWellbeingAlertCategory" \
,"AUCodeSetsWellbeingAppealStatus" \
,"AUCodeSetsWellbeingCharacteristicCategory" \
,"AUCodeSetsWellbeingCharacteristicClassification" \
,"AUCodeSetsWellbeingCharacteristicSubCategory" \
,"AUCodeSetsWellbeingEventCategoryClass" \
,"AUCodeSetsWellbeingEventLocation" \
,"AUCodeSetsWellbeingEventTimePeriod" \
,"AUCodeSetsWellbeingResponseCategory" \
,"AUCodeSetsWellbeingStatus" \
,"AUCodeSetsYearLevelCode" \
,"AUCodeSetsYesOrNoCategory" \
,"DefinedProtocols" \
,"GenericYesNo" \
,"ISO4217CurrencyNamesAndCodeElements"]


from multiprocessing.pool import ThreadPool
pool = ThreadPool(5)
pool.map(lambda e: PersistRefData(e,dict1),entities)

StatementMeta(adsdevsynspads, 1, 32, Finished, Available)

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]